<a href="https://colab.research.google.com/github/richfon/test_tyba/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FastAPI REST API: User Auth, Restaurant Search, Transactions

This notebook builds a REST API with FastAPI that supports:
- User registration, login, and logout
- Protected endpoint to search for nearby restaurants (by city or coordinates) using a public API
- Endpoint to list all transactions

In [24]:
# Install required packages
!pip install fastapi uvicorn[standard] python-multipart passlib[bcrypt] sqlalchemy httpx

In [25]:
!pip install python-jose

In [26]:
# Import libraries
from fastapi import FastAPI, Depends, HTTPException, status, Request
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from fastapi.middleware.cors import CORSMiddleware
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, Session
from passlib.context import CryptContext
from jose import JWTError, jwt
from datetime import datetime, timedelta
import httpx

### Database Setup

This cell configures the SQLite database using SQLAlchemy, sets up the engine, session, and base class for models.

In [27]:
# Database setup (SQLite for simplicity)
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

<ipython-input-27-d2ddd6559803>:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Define Database Models

This cell defines the `User` and `Transaction` models for storing user credentials and transaction logs in the database.

In [28]:
# Models
class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True, index=True)
    hashed_password = Column(String)

class Transaction(Base):
    __tablename__ = "transactions"
    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer, ForeignKey("users.id"))
    query = Column(String)
    timestamp = Column(DateTime, default=datetime.utcnow)
    user = relationship("User")

### Create Database Tables

This cell creates the tables in the SQLite database based on the defined models.

In [29]:
# Create tables
Base.metadata.create_all(bind=engine)

### Authentication and Utility Functions

This cell sets up password hashing, JWT token creation, and helper functions for user authentication and retrieval.

In [30]:
# Auth and utility functions
SECRET_KEY = "your-secret-key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)

def get_password_hash(password):
    return pwd_context.hash(password)

def create_access_token(data: dict, expires_delta: timedelta = None):
    to_encode = data.copy()
    expire = datetime.utcnow() + (expires_delta or timedelta(minutes=15))
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def get_user(db, username: str):
    return db.query(User).filter(User.username == username).first()

def authenticate_user(db, username: str, password: str):
    user = get_user(db, username)
    if not user or not verify_password(password, user.hashed_password):
        return False
    return user

def get_current_user(db: Session = Depends(get_db), token: str = Depends(oauth2_scheme)):
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"},
    )
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if username is None:
            raise credentials_exception
    except JWTError:
        raise credentials_exception
    user = get_user(db, username)
    if user is None:
        raise credentials_exception
    return user

### FastAPI App Setup

This cell initializes the FastAPI app and configures CORS middleware to allow cross-origin requests.

In [31]:
# FastAPI app setup
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [32]:
# User registration endpoint
@app.post("/register")
def register(form_data: OAuth2PasswordRequestForm = Depends(), db: Session = Depends(get_db)):
    user = get_user(db, form_data.username)
    if user:
        raise HTTPException(status_code=400, detail="Username already registered")
    hashed_password = get_password_hash(form_data.password)
    new_user = User(username=form_data.username, hashed_password=hashed_password)
    db.add(new_user)
    db.commit()
    db.refresh(new_user)
    return {"msg": "User registered successfully"}

In [33]:
# User login endpoint
@app.post("/token")
def login(form_data: OAuth2PasswordRequestForm = Depends(), db: Session = Depends(get_db)):
    user = authenticate_user(db, form_data.username, form_data.password)
    if not user:
        raise HTTPException(status_code=400, detail="Incorrect username or password")
    access_token = create_access_token(data={"sub": user.username}, expires_delta=timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES))
    return {"access_token": access_token, "token_type": "bearer"}

In [34]:
# User logout endpoint (stateless JWT, so just client-side token removal)
@app.post("/logout")
def logout():
    return {"msg": "Logout successful. Remove token on client side."}

### Protected Restaurant Search Endpoint

This cell implements the `/restaurants` endpoint, which allows authenticated users to search for nearby restaurants by city or coordinates using public APIs.

In [35]:
# Protected endpoint: Get nearby restaurants by city or coordinates
@app.get("/restaurants")
async def get_restaurants(
    city: str = None,
    lat: float = None,
    lon: float = None,
    current_user: User = Depends(get_current_user),
    db: Session = Depends(get_db)
):
    if city:
        # Use Nominatim to get coordinates from city name
        async with httpx.AsyncClient() as client:
            resp = await client.get(f"https://nominatim.openstreetmap.org/search", params={"q": city, "format": "json"})
            data = resp.json()
            if not data:
                raise HTTPException(status_code=404, detail="City not found")
            lat, lon = float(data[0]["lat"]), float(data[0]["lon"])
    if lat is None or lon is None:
        raise HTTPException(status_code=400, detail="Provide city or coordinates")
    # Use OpenStreetMap Overpass API to find restaurants
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node
      [amenity=restaurant]
      (around:3000,{lat},{lon});
    out;
    """
    async with httpx.AsyncClient() as client:
        resp = await client.post(overpass_url, data=query)
        restaurants = resp.json().get("elements", [])
    # Log transaction
    db.add(Transaction(user_id=current_user.id, query=f"{lat},{lon}"))
    db.commit()
    return [{"name": r.get("tags", {}).get("name", "Unknown"), "lat": r["lat"], "lon": r["lon"]} for r in restaurants]

In [36]:
# Endpoint to list all transactions
@app.get("/transactions")
def list_transactions(current_user: User = Depends(get_current_user), db: Session = Depends(get_db)):
    txs = db.query(Transaction).filter(Transaction.user_id == current_user.id).all()
    return [{"id": t.id, "query": t.query, "timestamp": t.timestamp.isoformat()} for t in txs]

### Run the FastAPI App

This cell provides the code to run the FastAPI app locally using Uvicorn.

In [ ]:
# Run the FastAPI app (for local testing)
# In Colab, use: !uvicorn filename:app --reload --port 8000
if __name__ == "__main__":
    import uvicorn
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [837] using WatchFiles


guide: https://stackoverflow.com/questions/60819376/fastapi-throws-an-error-error-loading-asgi-app-could-not-import-module-api

In [23]:
!uvicorn filename:app --reload --port 8000

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [25441] using WatchFiles
ERROR:    Error loading ASGI app. Could not import module "filename".
INFO:     Stopping reloader process [25441]
